In [1]:
import pandas as pd
from langdetect import detect
import glob


In [7]:
part1= pd.read_table("./train.tsv")
part2= pd.read_table("./valid.tsv",header=None)
part2.columns=part1.columns

In [8]:
part1.head()

,Id,Text,Label
0,1241490299215634434,Official death toll from #covid19 in the Unite...,INFORMATIVE
1,1245916400981381130,"Dearest Mr. President @USER 1,169 coronavirus ...",INFORMATIVE
2,1241132432402849793,Latest Updates March 20 ⚠️5274 new cases and 3...,INFORMATIVE
3,1236107253666607104,真把公主不当干部 BREAKING: 21 people on Grand Princess...,INFORMATIVE
4,1239673817552879619,OKLAHOMA CITY — The State Department of Educat...,UNINFORMATIVE


In [9]:
part2.head()

,Id,Text,Label
0,1241728922192142336,For those saying Pakistan isn’t Italy; After 3...,UNINFORMATIVE
1,1235713405992030209,Second case DR 🇩🇴 The Canadian woman has not b...,INFORMATIVE
2,1245941302367305728,Kill Chain: the cyber war on America's electio...,UNINFORMATIVE
3,1245913002840391681,Town hosts FIRST #Virtual #TownCouncil meeting...,UNINFORMATIVE
4,1240543259299987457,Report suggested that the actual number of und...,UNINFORMATIVE


In [10]:
main_df=pd.concat([part1,part2])

In [11]:
# main_df=main_df.drop(columns=["Id"])
main_df=main_df.drop_duplicates()
len(main_df)

7929

In [12]:
main_df

,Id,Text,Label
0,1241490299215634434,Official death toll from #covid19 in the Unite...,INFORMATIVE
1,1245916400981381130,"Dearest Mr. President @USER 1,169 coronavirus ...",INFORMATIVE
2,1241132432402849793,Latest Updates March 20 ⚠️5274 new cases and 3...,INFORMATIVE
3,1236107253666607104,真把公主不当干部 BREAKING: 21 people on Grand Princess...,INFORMATIVE
4,1239673817552879619,OKLAHOMA CITY — The State Department of Educat...,UNINFORMATIVE
...,...,...,...
995,1245955124222099456,Coronavirus took hold in UK earlier than thoug...,UNINFORMATIVE
996,1241768801210904576,I talked with a man who is Rowan County’s seco...,INFORMATIVE
997,1241172153040502795,Governor Wolf delaying enforcement of non-life...,UNINFORMATIVE
998,1239740620194766848,The Sheriff's Department has reduced the jail ...,UNINFORMATIVE


In [13]:
### Keep only english tweets

def is_english(text):

    try:
          language = detect(text)
    except:
          #These will be removed
          print("This row throws and error:", text)
          return False;
    if(language=='en'):
      return True;
    else:
      return False;

main_df=main_df[main_df.apply(lambda x: is_english(x['Text']), axis=1)]
len(main_df)

7929

In [14]:
main_df=main_df[["Id", "Label","Text"]]

In [15]:
main_df

,Id,Label,Text
0,1241490299215634434,INFORMATIVE,Official death toll from #covid19 in the Unite...
1,1245916400981381130,INFORMATIVE,"Dearest Mr. President @USER 1,169 coronavirus ..."
2,1241132432402849793,INFORMATIVE,Latest Updates March 20 ⚠️5274 new cases and 3...
3,1236107253666607104,INFORMATIVE,真把公主不当干部 BREAKING: 21 people on Grand Princess...
4,1239673817552879619,UNINFORMATIVE,OKLAHOMA CITY — The State Department of Educat...
...,...,...,...
995,1245955124222099456,UNINFORMATIVE,Coronavirus took hold in UK earlier than thoug...
996,1241768801210904576,INFORMATIVE,I talked with a man who is Rowan County’s seco...
997,1241172153040502795,UNINFORMATIVE,Governor Wolf delaying enforcement of non-life...
998,1239740620194766848,UNINFORMATIVE,The Sheriff's Department has reduced the jail ...


In [16]:
main_df.columns=["id", "Informativeness","text"]

In [17]:
main_df['Informativeness']=main_df['Informativeness'].map(lambda x: 1 if(x=='INFORMATIVE') else 0)

In [18]:
main_df.Informativeness.value_counts()

0    4187
1    3742
Name: Informativeness, dtype: int64

In [19]:
main_df.to_pickle("./../dataset4Cleaned.pkl")

In [2]:
main_df=pd.read_pickle("./../dataset4Cleaned.pkl")

In [3]:
from sentistrength import PySentiStr
senti = PySentiStr()
senti.setSentiStrengthPath('/Users/gerald/Workspaces/MA3/Semester Project/sula_master/SentiStrenght_files/SentiStrength.jar') # Note: Provide absolute path instead of relative path
senti.setSentiStrengthLanguageFolderPath('/Users/gerald/Workspaces/MA3/Semester Project/sula_master/SentiStrenght_files/SentStrength_Data_Sept2011/')
result = senti.getSentiment('stupid idiot', score='dual')
print(result)

def get_sentiment_score(x):
    result = senti.getSentiment(x["text"], score='dual')[0]
    x["positive_score"]=result[0]
    x["negative_score"]=result[1]
    x["emotional_devergence_score"]=(result[0]-result[1])/10.0
    
    return x

[(1, -3)]


In [4]:
from tqdm import tqdm
tqdm.pandas()
main_df = main_df.progress_apply(get_sentiment_score, axis=1)
main_df.to_pickle("./../dataset4Cleaned.pkl")

/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 7929/7929 [38:16<00:00,  3.45it/s]  
